In [1]:
import json

In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
data_dir = '/home/ubuntu/cfq/data/cfq_original'

In [4]:
%time dataset = json.load(open(f'{data_dir}/dataset.json'))

CPU times: user 1min 17s, sys: 19.4 s, total: 1min 37s
Wall time: 1min 36s


In [5]:
set(tuple((k, type(v)) for k, v in r.items()) for r in dataset)

{(('complexityMeasures', dict),
  ('expectedResponse', str),
  ('expectedResponseWithMids', str),
  ('question', str),
  ('questionPatternModEntities', str),
  ('questionWithBrackets', str),
  ('questionWithMids', str),
  ('ruleIds', list),
  ('ruleTree', dict),
  ('sparql', str),
  ('sparqlPattern', str),
  ('sparqlPatternModEntities', str))}

In [10]:
for i, r in enumerate(dataset):  # TODO: drop dicts
    r['index'] = i
    r['questionTemplate'] = r['complexityMeasures']['questionTemplate']

In [11]:
%time df = sc.parallelize(dataset).toDF().persist()

CPU times: user 42.7 s, sys: 8.67 s, total: 51.3 s
Wall time: 55.1 s


In [12]:
df.sort('index').rdd.map(lambda r: r['question']).take(10)

["Did  Jackie's female actor edit and produce Rad Plaid",
 "Did !Women Art Revolution's writer, director, editor, and star direct, produce, and write Conceiving Ada",
 'Did #FMV Flymen Vision employ a spouse of a character',
 "Did 'Gog' Helen''s writer, editor, cinematographer, producer, and director marry and influence Jacques Demy",
 "Did 'Murder' Legendre's male actor marry Lillian Lugosi",
 "Did (Buried) Over the Roofs' Spanish executive producer edit Shepherdesses",
 "Did ...And Justice for All's Canadian director executive produce Mokhtar",
 "Did 1 Night in Paris' male cinematographer's Canadian spouse's spouse marry Mayte Garcia",
 "Did 1/3/10's cinematographer and star edit, direct, and write Nakounine, 78 RPM, and The Subject of the Picture",
 "Did 1/3/10's star, director, editor, cinematographer, and writer write and direct Photographic Memory"]

In [13]:
[r['question'] for r in dataset[:10]]

["Did  Jackie's female actor edit and produce Rad Plaid",
 "Did !Women Art Revolution's writer, director, editor, and star direct, produce, and write Conceiving Ada",
 'Did #FMV Flymen Vision employ a spouse of a character',
 "Did 'Gog' Helen''s writer, editor, cinematographer, producer, and director marry and influence Jacques Demy",
 "Did 'Murder' Legendre's male actor marry Lillian Lugosi",
 "Did (Buried) Over the Roofs' Spanish executive producer edit Shepherdesses",
 "Did ...And Justice for All's Canadian director executive produce Mokhtar",
 "Did 1 Night in Paris' male cinematographer's Canadian spouse's spouse marry Mayte Garcia",
 "Did 1/3/10's cinematographer and star edit, direct, and write Nakounine, 78 RPM, and The Subject of the Picture",
 "Did 1/3/10's star, director, editor, cinematographer, and writer write and direct Photographic Memory"]

In [14]:
df.write.parquet(f'{data_dir}/dataset.parquet', mode='overwrite')